In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)


In [12]:
from openai import OpenAI
from IPython.display import Audio
import os

# Initialize the OpenAI client
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

VOICELINE = "The time is 10:23 AM."

# Generate speech with the 'nova' voice
response = client.audio.speech.create(
    model="tts-1",
    voice="nova",
    input=VOICELINE,
    response_format="wav",
)

In [14]:
import io
import soundfile as sf
import librosa

audio_bytes = response.content
with io.BytesIO(audio_bytes) as buf:
    buf.seek(0)
    data, samplerate = sf.read(buf, dtype="float32")
# Resample if necessary
if samplerate != 24000:
    data = librosa.resample(data.T, orig_sr=samplerate, target_sr=24000)

In [15]:
# Display in an audio widget
Audio(data, rate=24000)

In [18]:
from io import BytesIO

pseudo_file = BytesIO()
sf.write(pseudo_file, data, 24_000, format="wav")
pseudo_file.seek(0)
pseudo_file.name = "speech.wav"

transcript = client.audio.transcriptions.create(
    model="gpt-4o-mini-transcribe",
    file=pseudo_file,
)
print(transcript.text)

The time is 10:23 AM.


## Grading

In [22]:
from jiwer import wer, compute_measures

hypothesis = "It is twelve forty seven pm mm"
error = wer(VOICELINE, transcript.text)  # returns a float between 0 and potentially >1
measures = compute_measures([VOICELINE], [transcript.text])
print(measures)
# measures: {'wer': 0.2, 'substitutions': 1, 'deletions': 0, 'insertions': 1, 'hits': 4}

{'wer': 0.0, 'mer': 0.0, 'wil': 0.0, 'wip': 1.0, 'hits': 5, 'substitutions': 0, 'deletions': 0, 'insertions': 0, 'ops': [[AlignmentChunk(type='equal', ref_start_idx=0, ref_end_idx=5, hyp_start_idx=0, hyp_end_idx=5)]], 'truth': [['The', 'time', 'is', '10:23', 'AM.']], 'hypothesis': [['The', 'time', 'is', '10:23', 'AM.']]}


In [23]:
measures

{'wer': 0.0,
 'mer': 0.0,
 'wil': 0.0,
 'wip': 1.0,
 'hits': 5,
 'substitutions': 0,
 'deletions': 0,
 'insertions': 0,
 'ops': [[AlignmentChunk(type='equal', ref_start_idx=0, ref_end_idx=5, hyp_start_idx=0, hyp_end_idx=5)]],
 'truth': [['The', 'time', 'is', '10:23', 'AM.']],
 'hypothesis': [['The', 'time', 'is', '10:23', 'AM.']]}